In [1]:
import os
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd
import jpype
from konlpy.tag import Kkma
from hanspell import spell_checker
from pykospacing import spacing

In [2]:
# 파일 불러오기
df = pd.read_csv('ChatbotData.csv')
df_Q=df['Q']
df_A=df['A']
print(df_Q.head(),df_A.head())
df.info()

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object 0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11824 entries, 0 to 11823
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11824 non-null  object
 1   A       11824 non-null  object
 2   label   11824 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [3]:
# Series -> list
list_Q_A=[]

for i in range(len(df_Q)):
    temp=[]
    temp.append(str(i+1))
    temp.append(df_Q.iloc[i])
    temp.append(df_A.iloc[i])
    
    list_Q_A.append(temp)
list_Q_A[:10]

[['1', '12시 땡!', '하루가 또 가네요.'],
 ['2', '1지망 학교 떨어졌어', '위로해 드립니다.'],
 ['3', '3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'],
 ['4', '3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'],
 ['5', 'PPL 심하네', '눈살이 찌푸려지죠.'],
 ['6', 'SD카드 망가졌어', '다시 새로 사는 게 마음 편해요.'],
 ['7', 'SD카드 안돼', '다시 새로 사는 게 마음 편해요.'],
 ['8', 'SNS 맞팔 왜 안하지ㅠㅠ', '잘 모르고 있을 수도 있어요.'],
 ['9', 'SNS 시간낭비인 거 아는데 매일 하는 중', '시간을 정하고 해보세요.'],
 ['10', 'SNS 시간낭비인데 자꾸 보게됨', '시간을 정하고 해보세요.']]

In [4]:
# 형태소 분석
kkma = Kkma()
filter_kkma = ['NNG', 'NNP','OL','VA','VV','VXV']

def tokenizer_kkma(doc):
    # 꼬꼬마 형태소 분석기가 자바 기반이어서 파이썬에서 자바함수들을 실행할 수 있는 명령어 (jpype) 를 써줘야한다.
    jpype.attachThreadToJVM()       
    token_doc = ["/".join(word) for word in kkma.pos(doc)]
    return token_doc

def tokenize_kkma_noun_verb(doc):
    jpype.attachThreadToJVM()
    token_doc = ["/".join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
    return token_doc

In [5]:
token_Q_A = [(tokenizer_kkma(row[1]), row[0]) for row in list_Q_A]
tagged_Q_A = [TaggedDocument(d,[c]) for d,c in token_Q_A]

c:\users\dongho\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: jpype._core.attachThreadToJVM is deprecated, use java.lang.Thread.attach instead
  import sys


In [6]:
# 모델 만들기
# cpu 몇 개 쓸 건지
import multiprocessing
# 내 컴에 있는 cpu 갯수 cores 에 저장
cores = multiprocessing.cpu_count()
# vector_size : 임베딩할 벡터 차원
# negaive : negative sampling
d2v_Q_A = doc2vec.Doc2Vec(
    vector_size = 100,
    alpha = 0.025,
    min_alpha = 0.025,
    hs = 1,
    negative = 0,
    dm = 0,
    dbow_words = 1,
    min_count = 1,
    workers = cores,
    seed = 0
)

# 단어 사전 만들기
d2v_Q_A.build_vocab(tagged_Q_A)
for epoch in range(4):
  # 모델 학습
  print(epoch)
  d2v_Q_A.train(tagged_Q_A,
                 total_examples = d2v_Q_A.corpus_count,
                 epochs = d2v_Q_A.epochs)
  d2v_Q_A.alpha -=0.0025
  d2v_Q_A.min_alpha = d2v_Q_A.min_alpha

0
1
2
3


In [7]:
def doc2_answer(input_question): #질문 입력시 doc2으로 답변함
  token_test = tokenizer_kkma(input_question)
  predict_vector = d2v_Q_A.infer_vector(token_test)
  result = d2v_Q_A.docvecs.most_similar([predict_vector],topn=1)
  return list_Q_A[int(result[0][0])-1][2]

In [8]:
input_question = input()

dm=doc2_answer(input_question)
print(dm)

뭘해도 안되는 놈인가..
사고의 확대와 축소
